In [1]:
# install google chrome
!wget https://dl.google.com/linux/linux_signing_key.pub
!sudo apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!sudo apt-get -y update
!sudo apt-get install -y google-chrome-stable

# install chromedriver
# !apt-get install -y qq unzip1
!wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/

# install selenium
!sudo apt install -y python3-selenium
!pip install selenium==3.141.0 > /dev/null

# import libraries
import time
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import time
import pandas as pd

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://flip.xyz/login')

flip_login = 'mirad1side@gmail.com'
flip_password = 'FDd3b!BAmc5WC77'

time.sleep(4.5)

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
login_email = driver.find_element(By.ID, 'email')
login_email.send_keys(flip_login)
login_password = driver.find_element(By.ID, 'password')
login_password.send_keys(flip_password)
driver.find_element(By.XPATH, "//div[@class='mt-16']/button[1]").click()

data = pd.read_csv('../input/wallet-twitter/parsed_all_data_raw.csv', on_bad_lines='skip', sep=';')
data = data.dropna()
data_cleaned = data.drop(columns=['updated', 'pfp'])

i = 1
st = i*4400 + 1
stp = i*4400 + 4400

address_1 = [str(x) for x in data_cleaned.address.iloc[st:stp].to_numpy()]
address_perform_1 = {k: [] for k in address_1}
df = pd.DataFrame(list(address_perform_1.items()),columns = ['wallet', 'perform']) 

--2022-11-25 23:31:26--  https://dl.google.com/linux/linux_signing_key.pub
Resolving dl.google.com (dl.google.com)... 173.194.211.136, 173.194.211.190, 173.194.211.91, ...
Connecting to dl.google.com (dl.google.com)|173.194.211.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12415 (12K) [application/octet-stream]
Saving to: ‘linux_signing_key.pub’

linux_signing_key.p 100%[===================>]  12.12K  --.-KB/s    in 0s      

2022-11-25 23:31:26 (38.8 MB/s) - ‘linux_signing_key.pub’ saved [12415/12415]

OK
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1811 B]
Get:2 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5389 B]
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [6751 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:5 https://packages.cloud.google.com/apt google-fast-socket InRelease [5405 B]
Get:6 http://security.ubuntu.com/ubuntu focal-security

In [2]:
import pandas as pd
import requests

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
def summ(add):
    try:
        driver.get(f'https://flip.xyz/art/wallets/{add}/summary')   
        time.sleep(4.5)
        summary = driver.find_elements(By.CSS_SELECTOR, 'div.text-gray-50')
        if summary[1].text == '':
            collections = 0
        else: 
            collections = int(summary[1].text)
        if summary[2].text == '':
            nfts = 0
        else: 
            nfts = int(summary[1].text)
        if summary[4].text == '':
            buying_power = 0
        else: 
            buying_power = float(re.search(r'\d+.\d+', summary[4].text).group(0))
        a = [collections, nfts, buying_power]
        return a
    except:
        pass

In [4]:
def perf(add):
    try:
        driver.get(f'https://flip.xyz/art/wallets/{add}/performance')
        time.sleep(4.5)
        if driver.find_elements(By.CSS_SELECTOR, 'p.text-green-500') == []:
            realized_profit = driver.find_elements(By.CSS_SELECTOR, 'p.text-red-300')[0].text
        else:
            realized_profit = driver.find_elements(By.CSS_SELECTOR, 'p.text-green-500')[0].text
    
        if driver.find_elements(By.CSS_SELECTOR, 'p.text-green-500') == []:
            average_profit = driver.find_elements(By.CSS_SELECTOR, 'p.text-red-300')[1].text
        else:
            average_profit = driver.find_elements(By.CSS_SELECTOR, 'p.text-green-500')[1].text
    
        a = [realized_profit, driver.find_elements(By.CSS_SELECTOR, 'p.text-orange-400')[0].text, average_profit]
        return a
    except:
        pass

In [5]:
import pandas as pd
import requests

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

df['perform'] = df.wallet.progress_apply(perf)
df.to_csv('perfwall.csv')  

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


100%|██████████| 4328/4328 [2:58:31<00:00,  2.47s/it] 
